In [6]:
# Useful starting lines
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Load the training data into feature matrix, class labels, and event ids:

In [7]:
from proj1_helpers import *
import implementations as imp
%run pre_processing.ipynb

In [8]:
train_path = 'C:\\Users\\MehdiZayene\\Desktop\\train.csv'
test_path = 'C:\\Users\\MehdiZayene\\Desktop\\test.csv'

In [9]:
(tx_1,y_1,ids_1), (tx_2,y_2,ids_2), (tx_3,y_3,ids_3),indexes1,indexes2,indexes3,indexes4,indexes5,indexes6 = process_data(train_path)

In [10]:
(xtest_1,ids_1), (xtest_2,ids_2), (xtest_3,ids_3) = process_test(test_path,indexes1,indexes2,indexes3,indexes4,indexes5,indexes6)

## Do your thing crazy machine learning thing here :) ...

In [11]:
w1 , l1 = imp.least_squares(y_1,tx_1)
w2 , l2 = imp.least_squares(y_2,tx_2)
w3 , l3 = imp.least_squares(y_3,tx_3)

## Generate predictions and save ouput in csv format for submission:

In [12]:
y_pred1 = predict_labels(w1, xtest_1)
y_pred2 = predict_labels(w2, xtest_2)
y_pred3 = predict_labels(w3, xtest_3)

In [13]:
y = np.concatenate((y_pred1,y_pred2))
y = np.concatenate((y,y_pred3))

In [14]:
ids = np.concatenate((ids_1,ids_2))
ids = np.concatenate((ids,ids_3))

In [15]:
OUTPUT_PATH = 'C:\\Users\\MehdiZayene\\Desktop\\result.csv'
create_csv_submission(ids, y,OUTPUT_PATH)